In [1]:
import nltk
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input
from keras.layers import Embedding 
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalMaxPool1D
from keras.layers import LSTM, Bidirectional

In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [4]:
dt = pd.read_csv('train.csv')

In [5]:
# english data set
df_en = dt.loc[dt['lang_abv'] == 'en']
df_en

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [23]:
#english x
x_en = df_en.premise.values +' '+ df_en.hypothesis.values
#english y 
y_en = df_en.label.values
# split train and test
X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(x_en, y_en, test_size=0.3, random_state=100)

In [24]:
X_train_en

array(['Crosethe Rue de Rivoli to the Palais-Royal, built for Car?\xaddi?\xadnal Richelieu as his Paris residence in 1639, and originally named Palais-Cardinal. Cardinal Richelieu was a wealthy man who worked diligently for the Catholic church.',
       "If I had chosen to be an actor, I should have been the greatest actor living!  I chose to become an actor, but I wasn't very good at it. ",
       'Poor Dave, she said. She was happy for Dave.', ...,
       'Troyes is also a center for shopping, with two outlet centers selling both French and international designer-name fashions and home accessories. There are three outlet centers in Troyes, all of which sell only French fashions.',
       'Children will enjoy the little steam train that loops around the bay to Le Crotoy in the summer. There is a steam train looping around the bay to Le Crotoy.',
       'They said that (1) agencies need to be able to design their procedures to fit their particular circumstances (e.g. The authors of the

In [25]:
X_train_tok = [word_tokenize(sentence) for sentence in X_train_en]
X_test_tok = [word_tokenize(sentence) for sentence in X_test_en]

In [19]:
X_train_tok

[['Crosethe',
  'Rue',
  'de',
  'Rivoli',
  'to',
  'the',
  'Palais-Royal',
  ',',
  'built',
  'for',
  'Car',
  '?',
  '\xaddi',
  '?',
  '\xadnal',
  'Richelieu',
  'as',
  'his',
  'Paris',
  'residence',
  'in',
  '1639',
  ',',
  'and',
  'originally',
  'named',
  'Palais-Cardinal',
  '.',
  'Cardinal',
  'Richelieu',
  'was',
  'a',
  'wealthy',
  'man',
  'who',
  'worked',
  'diligently',
  'for',
  'the',
  'Catholic',
  'church',
  '.'],
 ['If',
  'I',
  'had',
  'chosen',
  'to',
  'be',
  'an',
  'actor',
  ',',
  'I',
  'should',
  'have',
  'been',
  'the',
  'greatest',
  'actor',
  'living',
  '!',
  'I',
  'chose',
  'to',
  'become',
  'an',
  'actor',
  ',',
  'but',
  'I',
  'was',
  "n't",
  'very',
  'good',
  'at',
  'it',
  '.'],
 ['Poor',
  'Dave',
  ',',
  'she',
  'said',
  '.',
  'She',
  'was',
  'happy',
  'for',
  'Dave',
  '.'],
 ['This',
  'site',
  'provides',
  'information',
  'links',
  ',',
  'tools',
  ',',
  'and',
  'resources',
  'developed

In [8]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(X_train_tok, vector_size=100, window=10, min_count=5, sg=1, epochs=20, negative=10)

In [10]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words],dtype=object)
                         for ls in X_train_tok],dtype=object)
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words],dtype=object)
                         for ls in X_test_tok],dtype=object)

In [11]:
X_train_vect

array([array([[-0.24320465326309204, 0.5428819060325623, -0.07731745392084122,
               ..., -0.08117348700761795, 0.4118991196155548,
               -0.17656657099723816],
              [0.037911202758550644, 0.1507505178451538, -0.18754751980304718,
               ..., -0.5797281265258789, 0.32218068838119507,
               -0.43073397874832153],
              [-0.15149664878845215, 0.3211745023727417, -0.22745241224765778,
               ..., 0.05923647806048393, 0.04857894405722618,
               -0.06872477382421494],
              ...,
              [-0.5044261813163757, 0.28322815895080566, -0.04446488991379738,
               ..., -0.15307943522930145, 0.04343312978744507,
               0.5220229625701904],
              [-0.6418529152870178, -0.05896664783358574, 0.12824110686779022,
               ..., -1.2125173807144165, 0.27019378542900085,
               0.3057483434677124],
              [-0.03458787500858307, 0.19672884047031403, -0.15449629724025726,
         

In [12]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [13]:
# data preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

scaler =  MinMaxScaler()
scaler.fit(X_train_vect_avg)
X_train_en = scaler.transform(X_train_vect_avg)
X_test_en = scaler.transform(X_test_vect_avg)

In [14]:
# Naive Bayes Classifier
NBC_pipeline = Pipeline([
    ('sel', SelectKBest(chi2, k= 15)),  # feature selection
    ('learner', MultinomialNB())  # learning algorithm
])

NBC_pipeline.fit(X_train_en,y_train_en)
predictions = NBC_pipeline.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      1.00      0.51       703
           1       0.00      0.00      0.00       667
           2       0.00      0.00      0.00       691

    accuracy                           0.34      2061
   macro avg       0.11      0.33      0.17      2061
weighted avg       0.12      0.34      0.17      2061

Confusion matrix:
[[703   0   0]
 [667   0   0]
 [691   0   0]]


C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
#SVM
param_grid = [{'sel__k': [10, 15, 20, 50], 'learner__C': [0.1, 1, 10,15]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)
SVM_opt_search.best_params_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\eleon\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'learner__C': 15, 'sel__k': 50}

In [16]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.39      0.32      0.35       703
           1       0.34      0.48      0.40       667
           2       0.36      0.28      0.32       691

    accuracy                           0.36      2061
   macro avg       0.36      0.36      0.36      2061
weighted avg       0.36      0.36      0.36      2061

Confusion matrix:
[[228 302 173]
 [172 322 173]
 [187 310 194]]


In [17]:
# Random Forest
param_grid = [{'sel__k': [8, 10, 5], 'learner__n_estimators': [100, 50, 500]}]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', RandomForestClassifier(bootstrap = True))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=True).fit(X_train_en,y_train_en)
opt_search.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


{'learner__n_estimators': 50, 'sel__k': 5}

In [18]:
opt_predictions = opt_search.best_estimator_.predict(X_test_en)

print('Classification report:')
print(classification_report(y_test_en, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test_en, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.34      0.38      0.36       703
           1       0.33      0.31      0.32       667
           2       0.29      0.27      0.28       691

    accuracy                           0.32      2061
   macro avg       0.32      0.32      0.32      2061
weighted avg       0.32      0.32      0.32      2061

Confusion matrix:
[[264 199 240]
 [241 207 219]
 [273 229 189]]


## neural network

In [53]:
word_vectors = w2v_model.wv

In [55]:
MAX_NB_WORDS = len(word_vectors.key_to_index)
MAX_SEQUENCE_LENGTH = 200

In [ ]:
from keras.preprocessing.sequence import pad_sequences
